## 🔄 Ajuste do Diretório de Trabalho

Antes de carregar ou manipular arquivos, é importante garantir que estamos no diretório correto do projeto.  
O código abaixo verifica se o notebook está sendo executado a partir da pasta `notebooks`. Se for o caso, ele sobe um nível na hierarquia de diretórios para garantir que o diretório de trabalho seja a raiz do projeto.

Isso é útil para manter caminhos relativos consistentes ao acessar dados, scripts ou outros recursos do projeto.

📌 **Resumo do que o código faz:**
- Verifica se o diretório atual termina com `notebooks`.
- Se sim, volta uma pasta (para a raiz do projeto).
- Exibe o novo diretório de trabalho.


In [4]:
import os

# Verifica se o diretório de trabalho atual termina com 'notebooks'
if os.path.basename(os.getcwd()) == 'notebooks':
    # Se sim, sobe um nível de diretório para a pasta raiz do projeto
    os.chdir('..')

# Imprime o diretório de trabalho para confirmar que a mudança foi feita
print(f"Diretório de Trabalho Atual: {os.getcwd()}")

Diretório de Trabalho Atual: c:\Users\Carlo\Desktop\Portfolio\postech-challenge-ibov


In [5]:
import duckdb
import pandas as pd
import numpy as np
import lightgbm as lgb
import shap
from sklearn.model_selection import train_test_split # Usaremos para a divisão temporal
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns

# Importa nossas configurações de projeto
import src.config as config

# Helpers do Notebook
from IPython.display import display

# Configurações de estilo
sns.set_theme(style='whitegrid', palette='viridis')
plt.style.use("fivethirtyeight")
%matplotlib inline

print("✅ Bibliotecas para a Fase 3 carregadas com sucesso!")

✅ Bibliotecas para a Fase 3 carregadas com sucesso!


In [14]:
# Carregando os dados da Fase 2 do nosso banco de dados

print(f"Carregando dados da tabela 'features_completas' de: {config.DB_PATH}")

try:
    con = duckdb.connect(database=str(config.DB_PATH), read_only=True)
    # MUITO IMPORTANTE: Selecionar da nova tabela com todas as features
    df_model = con.execute("SELECT * FROM features_completas").fetchdf()
    con.close()

    # Configura a coluna 'data' como o índice para facilitar a divisão temporal
    df_model['data'] = pd.to_datetime(df_model['data'])
    df_model.set_index('data', inplace=True)

    print("\n✅ Dados para modelagem carregados com sucesso!")
    print("Estrutura do DataFrame:")
    df_model.info()

except Exception as e:
    print(f"❌ Ocorreu um erro ao carregar os dados: {e}")
    print("Verifique se o pipeline da Fase 1 e o notebook da Fase 2 foram executados com sucesso.")

Carregando dados da tabela 'features_completas' de: C:\Users\Carlo\Desktop\Portfolio\postech-challenge-ibov\data\mercados.duckdb

✅ Dados para modelagem carregados com sucesso!
Estrutura do DataFrame:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2975 entries, 2014-02-05 to 2025-07-03
Data columns (total 83 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   close_petroleo_brent             2975 non-null   float64
 1   close_petrobras                  2975 non-null   float64
 2   close_dolar                      2975 non-null   float64
 3   close_ibovespa                   2975 non-null   float64
 4   close_sp500                      2975 non-null   float64
 5   high_petroleo_brent              2975 non-null   float64
 6   high_petrobras                   2975 non-null   float64
 7   high_dolar                       2975 non-null   float64
 8   high_ibovespa                    2975 non-null   

Carregando dados da tabela 'features_completas' de: C:\Users\Carlo\Desktop\Portfolio\postech-challenge-ibov\data\mercados.duckdb


,close_petroleo_brent,close_petrobras,close_dolar,close_ibovespa,close_sp500,high_petroleo_brent,high_petrobras,high_dolar,high_ibovespa,high_sp500,...,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0,spread_petro_ibov,ret_dolar_1d,ret_ibov_1d,ret_diff_dolar_ibov,dia_da_semana,mes
0,97.379997,5.078847,2.4064,46624.0,1751.640015,98.260002,5.244103,2.4167,47148.0,1755.790039,...,46923.8,47946.502616,4.358993,0.353428,0.000109,-0.012637,-0.007240,-0.005398,2,2
1,97.839996,5.218397,2.3955,47738.0,1773.430054,98.830002,5.273482,2.4098,48034.0,1774.060059,...,47022.6,48228.912464,5.130777,0.796524,0.000109,-0.004530,0.023893,-0.028423,3,2
2,99.879997,5.306533,2.3795,48074.0,1797.020020,100.239998,5.343256,2.3910,48253.0,1798.030029,...,47109.6,48525.689037,6.011892,0.840515,0.000110,-0.006679,0.007038,-0.013718,4,2
3,100.059998,5.365290,2.3752,47711.0,1799.839966,100.550003,5.398341,2.3993,48136.0,1799.939941,...,47422.2,48501.106409,4.550217,0.633839,0.000112,-0.001807,-0.007551,0.005744,0,2
4,99.940002,5.493823,2.4062,48463.0,1819.750000,100.599998,5.534218,2.4137,48604.0,1823.540039,...,47722.0,48948.039477,5.138257,0.802193,0.000113,0.013051,0.015762,-0.002710,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2970,65.519997,31.209999,5.4771,136866.0,6173.069824,66.089996,31.450001,5.5024,137209.0,6187.680176,...,136692.6,137715.377864,1.496464,0.584769,0.000228,-0.014538,-0.001809,-0.012729,4,6
2971,65.110001,31.379999,5.4784,138855.0,6204.950195,65.820000,31.490000,5.5071,139103.0,6215.080078,...,137153.4,139133.669315,2.887671,0.929639,0.000226,0.000237,0.014532,-0.014295,0,6
2972,65.449997,31.490000,5.4302,139549.0,6198.009766,65.980003,31.570000,5.4676,139695.0,6210.779785,...,137630.2,140387.576550,4.006935,0.847939,0.000226,-0.008798,0.004998,-0.013796,1,7
2973,67.449997,32.049999,5.4588,139051.0,6227.419922,67.580002,32.240002,5.4874,140049.0,6227.600098,...,138287.0,140458.475812,3.140535,0.675917,0.000230,0.005267,-0.003569,0.008835,2,7
